# Checks of races data

In [1]:
# Autoreload allows the notebook to dynamically load code: if we update some helper functions *outside* of the notebook, we do not need to reload the notebook.
%load_ext autoreload
%autoreload 2

In [36]:
import pandas as pd
import numpy as np
import re
import unicodedata

## Initial Info

In [8]:
csv_file = "dataset/races.csv"
dataset = pd.read_csv(csv_file)
dataset.head()

,_url,name,points,uci_points,length,climb_total,profile,startlist_quality,average_temperature,date,position,cyclist,cyclist_age,is_tarmac,is_cobbled,is_gravel,cyclist_team,delta
0,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,0,sean-kelly,22.0,True,False,False,vini-ricordi-pinarello-sidermec-1986,0.0
1,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,1,gerrie-knetemann,27.0,True,False,False,norway-1987,0.0
2,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,2,rene-bittinger,24.0,True,False,False,NaN,0.0
3,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,3,joseph-bruyere,30.0,True,False,False,navigare-blue-storm-1993,0.0
4,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,4,sven-ake-nilsson,27.0,True,False,False,spain-1991,0.0


In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589865 entries, 0 to 589864
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   _url                 589865 non-null  object 
 1   name                 589865 non-null  object 
 2   points               589388 non-null  float64
 3   uci_points           251086 non-null  float64
 4   length               589865 non-null  float64
 5   climb_total          442820 non-null  float64
 6   profile              441671 non-null  float64
 7   startlist_quality    589865 non-null  int64  
 8   average_temperature  29933 non-null   float64
 9   date                 589865 non-null  object 
 10  position             589865 non-null  int64  
 11  cyclist              589865 non-null  object 
 12  cyclist_age          589752 non-null  float64
 13  is_tarmac            589865 non-null  bool   
 14  is_cobbled           589865 non-null  bool   
 15  is_gravel        

In [10]:
dataset.describe()

,points,uci_points,length,climb_total,profile,startlist_quality,average_temperature,position,cyclist_age,delta
count,589388.000000,251086.000000,589865.000000,442820.000000,441671.000000,589865.000000,29933.000000,589865.000000,589752.000000,589865.000000
mean,89.221635,74.601547,166776.180584,2330.469215,2.611611,1101.161178,21.731768,74.219491,28.486208,418.292794
std,54.435330,100.947962,64545.605664,1375.710722,1.491741,380.586928,5.884761,48.404023,3.855631,842.961596
min,18.000000,6.000000,1000.000000,2.000000,1.000000,115.000000,10.000000,0.000000,13.000000,-6906.000000
25%,50.000000,16.000000,152500.000000,1309.000000,1.000000,844.000000,17.000000,32.000000,26.000000,10.000000
50%,80.000000,60.000000,178200.000000,2255.000000,2.000000,988.000000,22.000000,70.000000,28.000000,156.000000
75%,100.000000,100.000000,203500.000000,3273.000000,4.000000,1309.000000,26.000000,112.000000,31.000000,624.000000
max,350.000000,800.000000,338000.000000,6974.000000,5.000000,2047.000000,36.000000,209.000000,56.000000,61547.000000


In [115]:
# Count numer of same values in every column

#dataset['_url'].value_counts()
#dataset['name'].value_counts()
#dataset['points'].value_counts()
#dataset['uci_points'].value_counts()
#dataset['length'].value_counts()
#dataset['climb_total'].value_counts()
#dataset['profile'].value_counts()
#dataset['startlist_quality'].value_counts()
#dataset['average_temperature'].value_counts()
#dataset['date'].value_counts()
#dataset['position'].value_counts()
#dataset['cyclist'].value_counts()
#dataset['cyclist_age'].value_counts()
#dataset['is_tarmac'].value_counts()
#dataset['is_cobbled'].value_counts()
#dataset['is_gravel'].value_counts()
#dataset['cyclist_team'].value_counts()
#dataset['delta'].value_counts()


cyclist_team
liberty-seguros-wurth-team-2005     8869
roompot-nederlandse-loterij-2018    8773
chazal-vetta-mbk-1993               8094
kondor-1979                         7895
kazakhstan-2019                     7701
                                    ... 
atala-ofmega-1988                   1259
finland-2016                        1236
south-africa-1993                   1174
denmark-2003                         216
quickstep-innergetic-2009              3
Name: count, Length: 91, dtype: int64

## Check on '_url' data

In [52]:
# For each data, check if '_url' object is in form name/year/stage
i=0
for index, row in dataset.iterrows():
    if pd.isnull(row['_url']):
        # Split url by / in name, year and stage
        url = row['_url'].split('/')
        # Check if name, year and stage are in url
        if len(url) != 3:
            i+=1
            print(row['_url'], i)
        else:
            name = url[0].replace('-', '')
            year = url[1]
            # Check is name contains only letters and year contains only digits
            if not name.isalpha() or year.isdigit():
                i+=1
                print(row['_url'], i)

## Check on 'name' data

In [71]:
# For each data, check if 'name' object contains any number, except for names containing 'E3'
for index, row in dataset.iterrows():
    if not pd.isnull(row['name']):
        if (row['name'] != 'E3-Prijs Harelbeke'
            and row['name'] != 'E3 Harelbeke'
            and row['name'] != 'E3 Prijs Vlaanderen'
            and row['name'] != 'E3 Saxo Bank Classic'
            and row['name'] != 'E3 Harelbeke'
            and row['name'] != 'E3 Prijs Vlaanderen - Harelbeke'
            and row['name'] != 'Record Bank E3 Harelbeke'
            and row['name'] != 'E3 BinckBank Classic'
            and row['name'] != 'E3 Saxo Classic'):
            if any(char.isdigit() for char in row['name']):
                print(row['_url'], row['name'])

## Check on 'points' data

In [77]:
# For each data, check if 'points' has a '.' in it
for index, row in dataset.iterrows():
    if not pd.isnull(row['points']):
        if not '.' in str(row['points']):
            print(row['_url'], row['points'])

In [81]:
# For each data, check if 'points' float64 data is a digit
for index, row in dataset.iterrows():
    if not pd.isnull(row['points']):
        # Delete last two char from 'points'
        points = str(row['points'])[:-2]

        if not points.isdigit():
            print(row['_url'], row['points'])

## Check on 'uci_points' data

In [78]:
# For each data, check if 'uci_points' has a '.' in it
for index, row in dataset.iterrows():
    if not pd.isnull(row['uci_points']):
        if not '.' in str(row['uci_points']):
            print(row['_url'], row['uci_points'])

In [82]:
# For each data, check if 'uci_points' float64 data is a digit
for index, row in dataset.iterrows():
    if not pd.isnull(row['uci_points']):
        # Delete last two char from 'uci_points'
        uci_points = str(row['uci_points'])[:-2]

        if not uci_points.isdigit():
            print(row['_url'], row['uci_points'])

## Check on 'length' data

In [79]:
# For each data, check if 'length' has .0 at the end
for index, row in dataset.iterrows():
    if not pd.isnull(row['length']):
        if not str(row['length']).endswith('.0'):
            print(row['_url'], row['length'])

tour-de-suisse/1992/stage-4 32200.000000000004
tour-de-suisse/1992/stage-4 32200.000000000004
tour-de-suisse/1992/stage-4 32200.000000000004
tour-de-suisse/1992/stage-4 32200.000000000004
tour-de-suisse/1992/stage-4 32200.000000000004
tour-de-suisse/1992/stage-4 32200.000000000004
tour-de-suisse/1992/stage-4 32200.000000000004
tour-de-suisse/1992/stage-4 32200.000000000004
tour-de-suisse/1992/stage-4 32200.000000000004
tour-de-suisse/1992/stage-4 32200.000000000004
amstel-gold-race/2009/result 258600.00000000003
amstel-gold-race/2009/result 258600.00000000003
amstel-gold-race/2009/result 258600.00000000003
amstel-gold-race/2009/result 258600.00000000003
amstel-gold-race/2009/result 258600.00000000003
amstel-gold-race/2009/result 258600.00000000003
amstel-gold-race/2009/result 258600.00000000003
amstel-gold-race/2009/result 258600.00000000003
amstel-gold-race/2009/result 258600.00000000003
amstel-gold-race/2009/result 258600.00000000003
amstel-gold-race/2009/result 258600.00000000003
am

In [85]:
# For each data, check if 'length' float64 data is a digit
for index, row in dataset.iterrows():
    if not pd.isnull(row['length']):
        # Delete '.'
        length = str(row['length']).replace('.', '')

        if not length.isdigit():
            print(row['_url'], row['length'])

## Check on 'climb_total' data

In [86]:
# For each data, check if 'climb_total' has .0 at the end
for index, row in dataset.iterrows():
    if not pd.isnull(row['climb_total']):
        if not str(row['climb_total']).endswith('.0'):
            print(row['_url'], row['climb_total'])

In [87]:
# For each data, check if 'climb_total' float64 data is a digit
for index, row in dataset.iterrows():
    if not pd.isnull(row['climb_total']):
        # Delete last two char from 'climb_total'
        climb_total = str(row['climb_total'])[:-2]

        if not climb_total.isdigit():
            print(row['_url'], row['climb_total'])

## Check on 'profile' data

In [89]:
# For each data, check if 'profile' has .0 at the end
for index, row in dataset.iterrows():
    if not pd.isnull(row['profile']):
        if not str(row['profile']).endswith('.0'):
            print(row['_url'], row['profile'])

In [90]:
# For each data, check if 'profile' float64 data is a digit
for index, row in dataset.iterrows():
    if not pd.isnull(row['profile']):
        # Delete last two char from 'profile'
        profile = str(row['profile'])[:-2]

        if not profile.isdigit():
            print(row['_url'], row['profile'])

## Check on 'startlist_quality' data

In [93]:
# For each data, check if 'startlist_quality' float64 data is a digit
for index, row in dataset.iterrows():
    if not pd.isnull(row['startlist_quality']):
        if not row['startlist_quality'].is_integer():
            print(row['_url'], row['startlist_quality'])   

## Check on 'average_temperature' data

In [94]:
# For each data, check if 'average_temperature' has .0 at the end
for index, row in dataset.iterrows():
    if not pd.isnull(row['average_temperature']):
        if not str(row['average_temperature']).endswith('.0'):
            print(row['_url'], row['average_temperature'])

In [95]:
# For each data, check if 'average_temperature' float64 data is a digit
for index, row in dataset.iterrows():
    if not pd.isnull(row['average_temperature']):
        # Delete last two char from 'average_temperature'
        average_temperature = str(row['average_temperature'])[:-2]

        if not average_temperature.isdigit():
            print(row['_url'], row['average_temperature'])

## Check on 'date' data

In [98]:
# For each data, check if 'date' is in the format yyyy-mm-dd hh:mm:ss
for index, row in dataset.iterrows():
    if not pd.isnull(row['date']):
        if not re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', row['date']):
            print(row['_url'], row['date'])

## Check on 'position' data

In [100]:
# For each data, check if 'position' data is a digit
for index, row in dataset.iterrows():
    if not pd.isnull(row['position']):
        if not row['position'].is_integer():
            print(row['_url'], row['position']) 

In [112]:
# For each '_url', check if there are all the 'position' from 0 to the max one after the other
urls = dataset['_url'].unique()
for url in urls:
    positions = dataset.loc[dataset['_url'] == url]['position']
    if not all(positions == np.arange(positions.max() + 1)):
        print(url, positions)

## Check on 'cyclist' data

In [ ]:
# For each data, check if 'cyclist' is in the cyclist.csv file
cyclist_csv_file = "dataset/cyclists.csv"
cyclist_dataset = pd.read_csv(cyclist_csv_file)

for index, row in dataset.iterrows():
    if not pd.isnull(row['cyclist']):
        if not row['cyclist'] in cyclist_dataset['_url'].values:
            print(row['_url'], row['cyclist'])

## Check on 'cyclist_age' data

In [96]:
# For each data, check if 'cyclist_age' has .0 at the end
for index, row in dataset.iterrows():
    if not pd.isnull(row['cyclist_age']):
        if not str(row['cyclist_age']).endswith('.0'):
            print(row['_url'], row['cyclist_age'])

In [97]:
# For each data, check if 'cyclist_age' float64 data is a digit
for index, row in dataset.iterrows():
    if not pd.isnull(row['cyclist_age']):
        # Delete last two char from 'cyclist_age'
        cyclist_age = str(row['cyclist_age'])[:-2]

        if not cyclist_age.isdigit():
            print(row['_url'], row['cyclist_age'])

## Check on 'is_tarmac' data

In [ ]:
# For each data, check if 'is_tarmac' is a boolean
for index, row in dataset.iterrows():
    if not pd.isnull(row['is_tarmac']):
        if not isinstance(row['is_tarmac'], bool):
            print(row['_url'], row['is_tarmac'])

## Check on 'is_cobbled' data

In [ ]:
# For each data, check if 'is_cobbled' is a boolean
for index, row in dataset.iterrows():
    if not pd.isnull(row['is_cobbled']):
        if not isinstance(row['is_cobbled'], bool):
            print(row['_url'], row['is_cobbled'])

## Check on 'is_gravel' data

In [ ]:
# For each data, check if 'is_gravel' is a boolean
for index, row in dataset.iterrows():
    if not pd.isnull(row['is_gravel']):
        if not isinstance(row['is_gravel'], bool):
            print(row['_url'], row['is_gravel'])

## Check on 'cyclist_team' data

In [114]:
# For each data, check if 'cyclist_team' is in the formato team-year, where the last four characters are digits
for index, row in dataset.iterrows():
    if not pd.isnull(row['cyclist_team']):
        if not re.match(r'.+-\d{4}', row['cyclist_team']):
            print(row['_url'], row['cyclist_team'])

In [116]:
# For each data, check if 'cyclist_team' has the last four characters as digits
for index, row in dataset.iterrows():
    if not pd.isnull(row['cyclist_team']):
        if not row['cyclist_team'][-4:].isdigit():
            print(row['_url'], row['cyclist_team'])

## Check on 'delta' data

In [ ]:
# For each data, check if 'delta' has .0 at the end
for index, row in dataset.iterrows():
    if not pd.isnull(row['delta']):
        if not str(row['delta']).endswith('.0'):
            print(row['_url'], row['delta'])

In [ ]:
# For each data, check if 'delta' float64 data is a digit
for index, row in dataset.iterrows():
    if not pd.isnull(row['delta']):
        # Delete last two char from 'delta'
        delta = str(row['delta'])[:-2]

        if not delta.isdigit():
            print(row['_url'], row['delta'])

# Plots of distributions